  # 1. Trabajo Práctico N°3: Introducción a ML"

### Carga de bibliotecas

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
import seaborn as sb

### Carga del dataset

In [2]:
df = pd.read_csv("datasets/traffic_dataset+features.csv", index_col=0, parse_dates=True, dayfirst= True)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'datasets/traffic_dataset+features.csv'

### Transformaciones 

Aquí incluir el código desarrollado en TP2 para obtener las series transformadas (logaritmos naturales de los índices y tasas de retorno).

# Trabajo Práctico N°3: Introducción al Aprendizaje Automático

## Introducción

En los prácticos 1 y 2 se analizaron las series originales y se les realizaron diversas transformaciones. 
A partir de las series preprocesadas, en este práctico se persigue el objetivo general de entrenar algunos modelos predictivos univariantes sencillos y luego comparar su desempeño. 

En esta introducción se describen muy brevemente algunas metodologías propias de series temporales que serán necesarias para responder las consignas. La idea es solo introducirnos en lo mínimo necesario para adaptar lo visto en _Introducción al Aprendizaje Automático_ al caso de Series Temporales. Se proveen varios links al blog: [machinelearningmastery.com](https://machinelearningmastery.com/) de Jason Brownlee, porque contiene explicaciones muy claras y bastante código de Python que seguramente les será muy útil.

En la literatura estadística estándar, la selección de modelos (equivalente a la selección de features y de hiperparámetros) por lo general se realiza incorporando solo los componentes (features, algunos de los cuales pueden ser features retardados) que se prueban como estadísticamente significativos (esta es la forma de evitar el sobreajuste). Luego para validar los modelos se realizan dos tareas, una de ellas es evaluar la bondad de ajuste (si lo que predice el modelo es similar a lo que se observa) y la otra es chequear si se cumplen los supuestos del modelo.  Dichos supuestos por lo general se refieren a los términos de error, por lo que se realizan pruebas y se analizan gráficos de los residuos. 

### División en conjuntos de entrenamiento y test

Como se estudia en la materia _Introducción al Aprendizaje Automático_, para entrenar modelos y seleccionar cuál es el más adecuado, se debe __particionar los datos (ejemplos) en conjuntos de entrenamiento, validación y test__; o bien en conjuntos de entrenamiento y test, si se emplea validación cruzada. La práctica habitual para construir dichos subconjuntos, es por medio de selección aleatoria, a fines de garantizar que los mismos provengan de una misma distribución y a su vez evitar el sobreajuste. En el caso de datos de series temporales, dicha estrategia no resulta adecuada porque un muestreo aleatorio rompe el ordenamiento de los datos por lo que se pierde la autocorrelación que justamente es lo que le dá sentido al análisis. Por lo tanto, para dividir los conjuntos de entrenamiento y test se debe acudir a otras metodologías. 

Existen dos procedimientos de partición que son los más empleados para datos de series de tiempo. La forma más secilla consiste en particionar el dataset en un punto fijo del tiempo de modo tal que los datos mas antiguos se usan para entrenar los modelos y los mas nuevos para test([Train-Test Split](https://machinelearningmastery.com/backtest-machine-learning-models-time-series-forecasting/)). Otro procedimiento, denominado en el mismo blog como [Walk-Forward Validation](https://machinelearningmastery.com/backtest-machine-learning-models-time-series-forecasting/), consiste en realizar cortes recursivos, por ejemplo a partir de una ventana temporal inicial tomar los $s$ primeros datos para pronosticar el valor de la variable en $s+1$, luego tomar los $s+1$ primeros datos para estimar el valor en $s+2$, etc (debe notarse que el conjunto que se usa para entrenamiento crece en cada paso). Existe también una variación del segundo procedimiento en la que se mantiene fijo el tamaño del conjunto de entrenamiento, por ejemplo se usan los primeros $s$ datos para pronosticar el valor en $s+1$ como antes, pero en el siguiente paso se dropea la primera observación y se emplean los datos desde la observación 2 hasta la $s+1$ para pronosticar el valor en $s+2$ (entonces en cada paso el conjunto de entrenamiento esá formado por $s$ datos). 

### Entrenamiento de modelos

Exploraremos algunos modelos propios de la literatura estadística pero empleando una metodología de entrenamiento mas ligada a aprendizaje automático.
En la literatura estadística de Series Temporales se trabaja tanto con estimaciones puntuales (el pronóstico es un único número) como en la construcción de bandas de pronóstico (un rango entre los que se espera que esté el valor pronosticado con cierta probabilidad) y en la selección automática de los modelos se suelen emplear "criterios de información" tales como el de Akaike, Schwarz, Hannan-Quiin. En cambio en aprendizaje automático por lo general solo se busca efectuar una estimación puntual y para la selección de modelos (tuneo de hiperparámetros) se usan métricas predictivas. 

##### Modelos base

Una vez particionados los datos, por lo general es una buena idea estimar algunos modelos muy sencillos que sirven de referencia. Mas aún en el caso de los índices financieros, donde hay múltiples agentes intentando realizar los mejores pronósticos para, en base a ello, obtener cuantiosas ganancias vía trading. 

Para las tasas de retorno, el modelo base que surge naturalmente es un proceso formado por una constante mas un término aleatorio, $r_t = \alpha + \varepsilon_t$, por lo que los pronósticos consistirían en el valor promedio de la serie([Average Forecast](https://otexts.com/fpp2/simple-methods.html)). 

Dado que $r_t = y_{t-1}- y_{t}$ (representando $y$ a los logaritmos naturales de los índices), se tiene que $y_t = \alpha + y_{t-1} + \varepsilon_t$, o sea que un modelo base adecuado para trabajar con los logaritmos naturales de los índices es una caminata aleatoria (random walk) con drift ($\alpha$). En una caminata aleatoria sin drift, el valor de un momento está detertminado por el del período anterior mas una perturbación aleatoria, por lo tanto la regla para hacer pronosticos para el momento t+1 es simplemente usar el valor del momento t ([Naive Forecast](https://machinelearningmastery.com/persistence-time-series-forecasting-with-python/)). Si la misma también incluye un drift (como en este caso), el valor de un momento determinado está dado por el valor del momento anterior mas una perturbación aleatoria, mas una constante (el drift). Entonces el drift tiende a dotar a las series de componentes de tendencia similares a una tendencia determinística. Debe notarse que si bien conceptuamente existe el drift, por los análisis que hicimos en los prácticos anteriores el mismo es un número muy cercano a cero (el retorno diario promedio es muy pequeño en todos los índices).

##### Modelos ARIMA

Los modelos [Autorregresivos Integrados de Medias Móviles](https://otexts.com/fpp2/non-seasonal-arima.html) (ARIMA) emplean la autocorrelación de las series para mejorar los pronósticos.  Por lo general se representan como $ARIMA(p, d, q)$, donde $p$ indica la cantidad de retardos de la series que se incluyen, $d$ es la cantidad de veces que hay que diferenciar la serie para lograr su estacionariedad y $q$ es la cantidad de retardos del término de perturbación que se incluye, o sea, si $y'$ es la serie diferenciada $d$ veces, el modelo que se entrena es:

$$y_t^{'} = c + \phi_1 y_{t-1}^{'} +...+\phi_q y_{t-q}^{'}  + \theta_1 \varepsilon_{t-1}+...+ \theta_1\varepsilon_{t-q} + \varepsilon_{t}$$

Es útil tener en cuenta que los modelos "baseline" para los retornos y para los logaritmos naturales de los índices son de tipo ARIMA(0,0,0) y ARIMA(0,1,0); respectivamente.

##### Otros modelos

Modelos ARIMA estacionales, Suavizado Exponencial, X-13ARIMA-SEATS, Modelos Espacio-Estado (Filtro de Kalman).

### Métricas

En scikit learn están implementadas algunas métricas útiles para problemas de regresión habitualmente usadas en series temporales, tales como como el [Error Cuadrático Medio](https://scikit-learn.org/stable/modules/model_evaluation.html#mean-squared-error), el [Error Absoluto Medio](https://scikit-learn.org/stable/modules/model_evaluation.html#mean-absolute-error) y el [Error Absoluto Mediano](https://scikit-learn.org/stable/modules/model_evaluation.html#median-absolute-error).
Por supuesto que existen otras métricas. En particular, en la seccion 6 del artículo de [Tae-Hwy (2007)](https://pdfs.semanticscholar.org/dcd3/4f5077aa96df43c814e436b12d2a33401e9a.pdf) se proponen las métricas MFTR y MCFD para la evaluación de pronósticos sobre retornos financieros.


## Consignas


A partir de las series transformadas (logaritmos naturales de los índices y tasas de retorno), se proponen actividades orientadoras, sin perjuicio de que se puedan incorporar análisis adicionales. Las actividades no están separadas en secciones para resaltar que no se trata de pasos aislados, por ejemplo la separación train-test se va haciendo en foma simultánea al entrenamiento de los modelos.

**Se pide:**

* Realizar la división entre datos de entrenamiento y test usando el procedimiento [Walk-Forward Validation](https://machinelearningmastery.com/backtest-machine-learning-models-time-series-forecasting/). Alternativamente se la puede implementar usando [timeSeriesSplit](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html).
* Entrenar al menos un modelo "baseline" ([Average Forecast](https://otexts.com/fpp2/simple-methods.html) y
[Naive Forecast](https://machinelearningmastery.com/persistence-time-series-forecasting-with-python/)) para los retornos y para los logaritmos de los índices.
* Entrenar los correspondientes modelos ARIMA.
* Identificar cuáles son los parámetros y los hiperparamétros de los modelos ARIMA. 
* Tunear los hiperparamétros usando varias métricas ([Error Cuadrático Medio](https://scikit-learn.org/stable/modules/model_evaluation.html#mean-squared-error), [Error Absoluto Medio](https://scikit-learn.org/stable/modules/model_evaluation.html#mean-absolute-error), [Error Absoluto Mediano](https://scikit-learn.org/stable/modules/model_evaluation.html#median-absolute-error)). Puede ser útil la siguiente [ayuda](https://machinelearningmastery.com/grid-search-arima-hyperparameters-with-python/).
* Evaluar el desempeño de los modelos. ¿Cómo podemos saber en un problema de regresión si un modelo es "bueno"?¿Se pudo superar a los modelos "baseline"?

**Opcional:**

* Recortar el dataset para que no incluya el período asociado a la pandemia COVID19.
* Construir bandas de confianza para los pronósticos (http://alkaline-ml.com/pmdarima/usecases/stocks.html).
* Entrenar algún modelos de los mencionado en la sección "Otros modelos".
* Implementar las métricas MFTR y MCFD sugeridas para la evaluación de pronósticos sobre retornos financieros en la sección 6 del artículo de [Tae-Hwy (2007)](https://pdfs.semanticscholar.org/dcd3/4f5077aa96df43c814e436b12d2a33401e9a.pdf).

## Fecha de entrega

- __Versión preliminar: TBD__

- __Version final: TBD__

## Condiciones de entrega

Realizar un informe en el cual se presenten los resultados y conclusiones del análisis realizado. El mismo debe estar pensado para un público técnico pero que desconoce los aspectos propios del problema a resolver tal como los compañeros de clase. Dicho informe puede ser un notebook, a condición de que en el mismo se efectúe un análisis escrito suficientemente detallado de los resultados. 
